# Visualizing Aircraft Use Based on WWII THOR Data

## Introduction
This notebook takes the Theatre History of Operations [(THOR) World War Two data](https://data.world/datamil/world-war-ii-thor-data/workspace/file?filename=THOR_WWII_DATA_CLEAN.csv) and visualizes it based on the top six aircraft that appear in the dataset. The goal for this notebook is to examine how certain WW2 aircraft were used by Allied air forces. We will use Pandas and Bokeh to sort and visualize a dataset. Filtering will help make the data easier to navagate, and visualizing the data will also make it easier to understand. We will also explore subsets of data and missing data gague the accuracy of our visualizations. Hopefully this makes the data more accessible, and the same methods can be used to explore other aspects of the dataset, helping to inspire further research and investigation.
___

## Data
The data we will use for the the notebook is THOR Data, which is a database of historical aerial bombings from World War One to Vietnam. The dataset we will be using combines digitized paper mission reports from World War Two, from 1939 to 1945. [Here](https://data.world/datamil/world-war-ii-thor-data) is a link to the full dataset, with a description and summary. 
<br><br>The dataset we will be using was reduced by Charlie Harper in his tutorial, [Vidualizing Data with Bokah and Pandas](https://programminghistorian.org/en/lessons/visualizing-with-bokeh#the-wwii-thor-dataset). The CSV file has been reduced to 19 columns from the original 62. 
<br><br>
Our data is limited because it only includes 19 out of the total 62 columns of data. However, this also allows us to narrow down our analysis to bombing data and core mission information. The dataset also sorts aircraft model information by type, which claims to be ["more robust than previous studies"](https://data.world/datamil/thor-data-dictionary/workspace/file?filename=THOR+Draft+Data+Dictionary+Dec+2016.pdf). For this reason, it is a relatively thorough record of the types of airplanes that were used by Allied forces in aerial bombings throughout the war (from Sept 3, 1939 to May 1945). However, the dataset contains more information about missions in the European theatre and is [less thorough on RAF boming in the Mediterranea, African, and Pacific theatres](https://data.world/datamil/thor-data-dictionary/workspace/file?filename=THOR+Draft+Data+Dictionary+Dec+2016.pdf). The data was also [compiled by an American team working with the Pentagon](https://insight.livestories.com/s/v2/thor-overview/a100cd16-c2a7-453b-8ea6-45947c1bbc51), so the data is skewed heavily with American sources. We will further investigate the limitations of the data for our purposes by calculating missing data from each column. 
<br><br> Using the code below, we can open up our dataset with Pandas to begin our exploration.

### Opening THOR_WW2 with Pandas
Now, we will use Pandas to read the CSV file and display the data in a chart. After the data is displayed, we can use the `.sample()` method to display 10 random rows of data.

In [ ]:
# importing pandas
import pandas as pd
# pandas is displaying the data up to 100 rows
pd.options.display.max_rows = 100
# panas is reading the thor_wwii csv file
df = pd.read_csv('thor_wwii.csv', delimiter=",", parse_dates=['MSNDATE'])
df

In [ ]:
#pandas is displaying 10 random rows of data from the set
df.sample(10)

___

## Sorting the THOR_WW2 data
To determine what kind of information we can examine and visualize in this dataset, we can print out a list of the columns in the CSV file using `df.columns.tolist()`.

In [ ]:
#showing a list of the column headings in the data
df.columns.tolist()

Seeing the list of columns in the THOR WW2 data, I want to focus on examining the aircrafts used in aerial bombings included in the dataframe. Using Pandas, we can sort by the country mission and aircraft name columns using double brackets `[[]]`.

In [ ]:
# displaying only the mission country, the aircraft names, and target countries
# this is the subset of data that I will be working with in this notebook
df[['COUNTRY_FLYING_MISSION', 'AIRCRAFT_NAME']]

Now, let's use `.unique()` to get a list of unique aircraft names within the dataframe. This way, we won't have to sort through all of the rows ourselves.

In [ ]:
#listing the unique aircraft names that appear in the dataset
df.AIRCRAFT_NAME.unique()

Since we have a list of all of the aircraft listed in the dataframe, let's see if we can narrow down our analysis by looking at the most frequently used aircrafts. We can do this by using `.value_counts()` and specifying the column AIRCRAFT_NAME.

In [ ]:
#counting the values of each aircraft name in the column
df['AIRCRAFT_NAME'].value_counts()

***

## Using Bokeh to visualize aircraft data
Now that we know the most reported aircraft in the dataframe, we can take the top six and use Bokeh to create interactive visualizations. It will be good to visualize the data because we will be able to see how the frequency of aircraft compares to one another. The interactive aspects of the charts also make this process easier since we can hover over the bars to compare the different types of munitions used. 
<br><br>
To start using Bokeh, we first have to install it into our notebook.

In [ ]:
# command to install bokeh
# the ! means that the code below is interpreted as being run on the command line, rather than as python
!pip install bokeh pyproj

### Bar charts of categorical data
Now that we've installed Bokeh, we can use it to create charts of categorical data. Categorical data is data that can be divided into different groups. Types of aircraft hold categorical data. Using a bar chart, we can compare how many munitions each plane dropped. The categorical data will go on our x axis while the numerical data (weight of munitions dropped) will be graphed on the y axis. This information can be used to compare how each aircraft was used.

In [ ]:
#bar graph visualization of the kilotons of munitions dropped by top 6 aircrafts that appear the most in the data
#importing bokeh charts and colours
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

from bokeh.palettes import Pastel1
from bokeh.transform import factor_cmap
output_notebook()

#selecting the top 6 aircrafts that were determined above
filter = df['AIRCRAFT_NAME'].isin(('B17','B24','B25','B26','A20','WELLINGTON'))
df = df[filter]

#grouping from the whole record to one record for each aircraft with the total munitions dropped
grouped = df.groupby('AIRCRAFT_NAME')['TOTAL_TONS', 'TONS_HE', 'TONS_IC', 'TONS_FRAG'].sum()
#grouping by 1000 gives us kilotons of munitions rather than just tons
grouped = grouped / 1000

#this creates a list of aircraft names to use, which is categorical instead of numerical
source = ColumnDataSource(grouped)
aircraft = source.data['AIRCRAFT_NAME'].tolist()
p = figure(x_range=aircraft)

color_map = factor_cmap(field_name='AIRCRAFT_NAME', 
                    palette=Pastel1[6], factors=aircraft)

#vbar makes a top parameter rather than a y parameter
p.vbar(x='AIRCRAFT_NAME', top='TOTAL_TONS', source=source, width=0.70, color=color_map)

# chart labels
p.title.text ='Munitions Dropped by Aircraft in the RAF and AF'
p.xaxis.axis_label = 'Aircraft'
p.yaxis.axis_label = 'Kilotons of Munitions'

#interactive hover tool that displays the amounts of munitions dropped for each column
hover = HoverTool()
hover.tooltips = [
    ("Totals", "@TONS_HE High Explosive / @TONS_IC Incendiary / @TONS_FRAG 	Fragmentation")]

#'vline' tells the popup to show when a vertial line crosses a glyph
hover.mode = 'vline'

p.add_tools(hover)

show(p)

To double check my data before coming to any conclusions, I am going to print the grouped information to see that all of the numbers make sense. The output below shows us that the tons of high explosive, incindiary, and fragmentation munitions all add up to the total tons of munitions overall for each aircraft, which means the visualization is showing an accurate representation of the data.

In [ ]:
print(grouped)

### Sub-sampling data
To get even more specific with the visualization, we can sub-sample the data and put it all together in a stacked bar chart. Examples of sub-samples in this context would be the different types of munitions used: high explosive, fragementation, and incendiary. Perhaps some aircrafts with less frequency in the data might have been used to drop a different kind of munition that was used less.

In [ ]:
#plotting the different types of munitions dropped by each aircraft
#importing bokeh charts, colours, and the hovertool
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool

from bokeh.palettes import Pastel1
output_notebook()

#selecting the top 6 aircrafts that were determined above
filter = df['AIRCRAFT_NAME'].isin(('B17','B24','B25','B26','A20','WELLINGTON'))
df = df[filter]

#grouping from the whole record to one record for each aircraft with the total munitions dropped
grouped = df.groupby('AIRCRAFT_NAME')['TONS_IC', 'TONS_FRAG', 'TONS_HE'].sum()
#grouping by 1000 gives us kilotons of munitions rather than just tons
grouped = grouped / 1000

#this creates a list of aircraft names to use, which is categorical instead of numerical
source = ColumnDataSource(grouped)
aircraft = source.data['AIRCRAFT_NAME'].tolist()
p = figure(x_range=aircraft)

#stacking the different kinds of munitions and setting the display colours
p.vbar_stack(stackers=['TONS_HE', 'TONS_FRAG', 'TONS_IC'], 
             x='AIRCRAFT_NAME', source=source, 
             legend_label = ['High Explosive', 'Fragmentation', 'Incendiary'],
             width=0.5, color=Pastel1[3])

# chart labels
p.title.text ='Types of Munitions Dropped by Top Planes'
p.legend.location = 'top_left'

p.xaxis.axis_label = 'Plane'
p.xgrid.grid_line_color = None	#remove the x grid lines

p.yaxis.axis_label = 'Kilotons of Munitions'

#interactive hover tool that displays the amounts of munitions dropped for each column
hover = HoverTool()
hover.tooltips = [
    ("Totals", "@AIRCRAFT_NAME Aircraft Name / @TONS_HE High Explosive / @TONS_IC Incendiary / @TONS_FRAG 	Fragmentation")]

hover.mode = 'vline'

p.add_tools(hover)

show(p)

The graph above suggests that all of the aircraft were used to drop high explosive munitions, which were also dropped most frequently by each kind of aircraft. It is interesting to note that aircrafts B17 and B24 were used most frequently, regardless of the type of munitions dropped. Why might this be? It also seems that the Wellington aircraft was rarely used to deploy incendiary munitions.

### Time-series visualization
Another way we can visualize the data is by using a time-series chart, which will graph the amount of munitions that the aircrafts collectively dropped over time. 
<br><br>
To get a set of data with the information we want, we will filter our dataframe to only show the information relating to the aircrafts B17, B24, B25, B26, A20, and Wellington. We will do this by using the same `filter` as we did above for the graphs. We will save that data as a new CSV file and then put that dataframe up on a time-series chart.

In [ ]:
#making a new dataset with the top 6 aircraft to make a time series visualization
filter = df['AIRCRAFT_NAME'].isin(('B17','B24','B25','B26','A20','WELLINGTON'))
df = df[filter]

df[filter]

In [ ]:
#writing our filter to CSV and saving
top_aircraft_filter = df[filter]
top_aircraft_filter.to_csv("THORwwii_top6aircrafts.csv", encoding='utf-8', index=False)

Now, we can use that CSV file for our time-series graph. We will resample the data by month using the `grouper` object and resampling by month `freq='M'`. We will be creating fewer records, which is called downsampling. This will make the chart look cleaner and hopefully allow us to see trends more easily.

In [ ]:
#time graph showing the types of munitions dropped by the top 6 aircraft over time
#importing bokeh charts and colours
import pandas as pd
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Pastel1
output_notebook()

#here the code is reading only the top aircraft data instead of the whole datafile which was abbreviated as df
topcraft_df = pd.read_csv('THORwwii_top6aircrafts.csv')

#make sure MSNDATE is a datetime format
topcraft_df['MSNDATE'] = pd.to_datetime(topcraft_df['MSNDATE'], format='%Y/%m/%d')

#resampling the data by month (M) so that we get a cleaner graph
grouped = topcraft_df.groupby(pd.Grouper(key='MSNDATE', freq='M'))['TOTAL_TONS', 'TONS_IC', 'TONS_FRAG', 'TONS_HE'].sum()
grouped = grouped/1000

source = ColumnDataSource(grouped)

#x axis type
p = figure(x_axis_type='datetime')

#setting colours for each line in the chart
p.line(x='MSNDATE', y='TOTAL_TONS', line_width=2, source=source, legend_label='All Munitions')
p.line(x='MSNDATE', y='TONS_FRAG', line_width=2, source=source, color='pink', legend_label='Fragmentation')
p.line(x='MSNDATE', y='TONS_IC', line_width=2, source=source, color='yellow', legend_label='Incendiary')
p.line(x='MSNDATE', y='TONS_HE', line_width=2, source=source, color='green', legend_label='High Explosive')

#y axis label
p.yaxis.axis_label = 'Kilotons of Munitions Dropped'

show(p)

Here, we can see that munitions use for this set of aircrafts increased significantly after 1943. There is a dip in the graph just before the winter of 1944, before Germany surrendered. There is an increase again in munitions dropped in the spring of 1945, leading up to the surrender of Japan in the fall of 1945. It is important to note as well that the use of all top six bombing aircraft was limited until 1943. 
***

## Examining missing data
One important thing to consider when doing broad examination of data is how much of that data you really have. With this dataset, we can use Python to calculate the percentage of complete data in the CSV file. The `.count()` method will exclude NaN values in the data. Using `len()` will give us the percentage of filled data in every column. <br>
Note that an "unknown" will still be counted if it has been included in the dataset. The `.count()` method only accounts for blank cells the columns. 

In [ ]:
#finding what % of each column is missing information
df.count() / len(df)

After running that cell, we can see that the AIRCRAFT_NAME column is 99% full. That gives us a good indication that the broad trends we discovered using the visualizations are accurate according to the data that was included in the THOR_WW2 dataset. This is true as well for our time-series chart because that MSNDATE column has 100% of its data. From these percentages we can also see that takeoff locations were less important to record than target locations. The data included in the set also suggests that the most important things to note were when the missions took place and how many munitions were being used by weight.
***

# Conclusion
Overall, we have been able to explore and visualize our THOR WW2 data. Using Pandas, we have sorted the data by mission country and aircraft name. We were able to determine how many times each type of aircraft appeared in the dataset. With this information, we were able to visualize the top six types of aircraft present in the dataset using Bokeh. Bokeh's interactive features allowed us to compare the aircraft, how often each were used, and the types of munitions that each aircraft dropped. We can see that most aircraft were used to deploy high explosive munitions, and aircrafts B17 and B24 were most used overall, regardless of munition type. Finally, we were also able to examine how much of the data was missing from our dataset, giving us insight as to what kind of information was deemed important enough to record. 
<br><br>
Further research could examine Canada's own contribution to WW2 aerial missions by crossreferencing with other data. For example, I considered creating a time-series chart with the [Ingenium data](https://ingeniumcanada.org/collection-research/artifact-open-data-set-mash-up), filtering for aircraft parts manufactured between 1939 to 1945. Since the data has robust information on the target countries of each mission, the same filtering and visualizations used in this notebook could be used to examine types of munitions used on target countries. Aircraft type by mission country could also be examined. Interesting results might also be yielded by exploring the least-used aircraft. Linking this data with other archival data, such as war diaries or photography catalogues, could also help to contextualize the categorical and quantitative data that we have worked with today, shifting focus to a more subjective and relationship-based exploration. 
<br><br>
The filtering and visualization methods used in this notebook could also be used for different datasets that contain quantitative, qualitative (categorical), and temporal data that could be compared to one another.
***

# References
THOR data information:
- [Description and summary of THOR WW2 data](https://data.world/datamil/world-war-ii-thor-data)
- [THOR data dictionary, with methodology, strenghts and limitations](https://data.world/datamil/thor-data-dictionary/workspace/file?filename=THOR+Draft+Data+Dictionary+Dec+2016.pdf)
<br>
- [An Overview of THOR](https://insight.livestories.com/s/v2/thor-overview/a100cd16-c2a7-453b-8ea6-45947c1bbc51); history behind the project

Tutorials followed:
<br>
- Pandas Basics [Part 1](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Data-Analysis/Pandas-Basics-Part1.html) and [Part 2](https://melaniewalsh.github.io/Intro-Cultural-Analytics/Data-Analysis/Pandas-Basics-Part2.html#count) by Melanie Walsh
- [Visualizing Data with Bokeh and Pandas](https://programminghistorian.org/en/lessons/visualizing-with-bokeh#the-wwii-thor-dataset) by Charlie Harper with The Programming Historian
<br>

Forums that helped with errors along the way:
- [list index out of range](https://stackoverflow.com/questions/40916388/python-pandas-bokeh-indexerror-list-index-out-of-range-why) error when creating bar charts
- [time data format does not match](https://stackoverflow.com/questions/25015711/time-data-does-not-match-format) error when creating a time-series chart
<br>

[Markdown guide](https://www.markdownguide.org/basic-syntax/)

# Further Reading
- [Data Stories: An Overview of THOR](https://insight.livestories.com/s/v2/thor-overview/a100cd16-c2a7-453b-8ea6-45947c1bbc51) 
<br>
This article discusses more informtion about how the THOR data was developped. It has notes on how the data can be used as an archive, for personal exploration, and for more applied purposes, like investigating unexploded ordnance. This article is very insightful as a starting point to be more informed on the scope of the data.
<br><br>
- [Data as Medium](https://medium.com/@caw_/data-as-medium-361814dba6a9) by Chad Weinard (2019)
<br>
This article does a great job exploring the subjectivity of data. Recording, cleaning, and exploring data is always an ongoing process that is profoundly shaped by human action. The dataset itself can be an object of study -- what does it say about the institution or the culture that produced it?
<br><br>
- [Archives strengthening historical narratives: Sharing digital and linked data resources for broader reach and sustainability](https://mw18.mwconf.org/paper/archives-strengthening-historical-narrative-sharing-digital-and-linked-data-resources-for-broader-reach-and-sustainability/) by Mark Coffey, Alan Watts, and Duane Degler (2018)
<br>
Since the THOR data is just raw information that is not presented in any particularly accessible way, this article presents and example of how archival materials can be made more accessible. The article breaks down an interactive website that allows users to explore the archive through many different points of entry that accommodate for different interests. This method also allows users to discover and explore further than what they may have intitally been looking for. The emphasis on linking data and sharing assets also applies to the THOR data, which can be used as a jumping off point for further investigation.